In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from open_spiel.python.examples.ubc_cma import *

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy, ActivityPolicy, UndersellPolicy, TiebreakingPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator

Loading BokehJS ...

In [3]:
# load runs from experiments
experiments = []
experiments += ['oct04_external_heuristics_v8']

runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")

Found 900 runs


In [9]:
runs[0].name

'oct4_encumberedheuristic_oct4_encumberedheuristic_3_base_tie_break-cfr_port_5_extexternal_plus-100'

In [20]:
run = EquilibriumSolverRun.objects.filter(
    name="oct4_encumberedheuristic_oct4_encumberedheuristic_3_spite_undersell_allowed-cfr_port_5_extexternal_plus-103",
    experiment__name="oct04_external_heuristics_v8"
)[0]

In [26]:
game, final_checkpoint, policy = get_results(run, load_policy=True)
evaluation = final_checkpoint.get_modal_eval()


In [44]:
state = game.new_initial_state().child(1).child(1)

In [45]:
state.information_state_string()

'p0t1 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]'

In [48]:
policy._infostates

{'p0t0 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]': [array([  0.        ,   0.        , 634.51113241]),
  array([1.0000000e-06, 7.6521850e+02, 6.0707815e+03]),
  0,
  0],
 'p1t0 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]': [array([ 0.       , 15.8207248]),
  array([2.84448285e+00, 2.04031555e+04]),
  0,
  0],
 'p1t1 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]': [array([   0.        ,   35.07218674, 1144.69502257]),
  array([1.00000000e-06, 1.71807836e+03, 1.82289216e+04]),
  0,
  0],
 'p0t0 r2 posted[[32.0, 19.0, 13.0], [32.0, 24.7, 16.9]] sub[[0, 0, 0], [0, 1, 1]] a32 agg[[0, 0, 0], [1, 2, 2]] proc[[0, 0, 0], [0, 1, 1]]': [array([ 0.       , 36.5467443]),
  array([  4.000001, 775.000001]),
  0,
  0],
 'p1t1 r2 posted[[32.0, 19.0, 13.0], [32.0, 24.7, 16.9]] sub[[0, 0, 0], [1, 1, 1]] a64 agg[[0, 0, 0], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1]]': [array([ 0.        ,  0.        , 82.22100216]),
  arra

In [46]:
policy._infostates[state.information_state_string()]

[array([   0.        , 1497.7647941 ,   61.22683889]),
 array([1.04977533e+00, 4.94666381e+03, 1.66728642e+03]),
 0,
 0]

In [51]:
policy._infostates

{'p0t0 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]': [array([  0.        ,   0.        , 634.51113241]),
  array([1.0000000e-06, 7.6521850e+02, 6.0707815e+03]),
  0,
  0],
 'p1t0 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]': [array([ 0.       , 15.8207248]),
  array([2.84448285e+00, 2.04031555e+04]),
  0,
  0],
 'p1t1 r1 posted[[32, 19, 13]] sub[[0, 0, 0]] a64 agg[[0, 0, 0]] proc[[0, 0, 0]]': [array([   0.        ,   35.07218674, 1144.69502257]),
  array([1.00000000e-06, 1.71807836e+03, 1.82289216e+04]),
  0,
  0],
 'p0t0 r2 posted[[32.0, 19.0, 13.0], [32.0, 24.7, 16.9]] sub[[0, 0, 0], [0, 1, 1]] a32 agg[[0, 0, 0], [1, 2, 2]] proc[[0, 0, 0], [0, 1, 1]]': [array([ 0.       , 36.5467443]),
  array([  4.000001, 775.000001]),
  0,
  0],
 'p1t1 r2 posted[[32.0, 19.0, 13.0], [32.0, 24.7, 16.9]] sub[[0, 0, 0], [1, 1, 1]] a64 agg[[0, 0, 0], [1, 2, 2]] proc[[0, 0, 0], [1, 1, 1]]': [array([ 0.        ,  0.        , 82.22100216]),
  arra

In [47]:
(1497) / (1497 + 61)

0.9608472400513479